In [ ]:
!wget https://github.com/CSSEGISandData/COVID-19/archive/master.zip
!unzip master.zip

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import os
from skimage import io

import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
root_path = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'

recent = sorted(os.listdir(root_path))[-2]
path = os.path.join(root_path, recent)
df = pd.read_csv(path)
print(df.shape)
display(df)
country = df['Country/Region'].to_numpy()
print(np.unique(country))

In [ ]:
dic_coor = {}
for index, row in df.iterrows():
  dic_coor[row['Country/Region']] = [row['Longitude'], row['Latitude']]

In [ ]:
# Carregar mapa
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
for tipo in ['abs', 'log']:
  print("Gerando dados tipo", tipo, "\n")

  if not os.path.isdir(tipo):
    os.mkdir(tipo)

  for k, csv in enumerate(sorted(os.listdir(root_path))):
    plt.close('all')
    print(csv[:-4], end=',')
    
    if not csv[-3:] == 'csv': continue
    
    path = os.path.join(root_path, csv)
    df = pd.read_csv(path)
    
    # Converter latitude e longitude para GeoDataFrame
    if 'Longitude' not in df.columns.to_numpy():

      regions = df['Country/Region']
      regions = [r if r in dic_coor else 'China' for r in regions]
      longitude = [dic_coor[r][0] for r in regions]
      latitude  = [dic_coor[r][1] for r in regions]

      df['Longitude'] = longitude
      df['Latitude']  = latitude  
      
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'],df['Latitude']))

    ax = world.plot(figsize=(15, 8))
    
    if tipo == "log":
      gdf.plot(ax=ax, color='darkred', markersize=np.log10(gdf['Confirmed'])*2e1, alpha=0.6, ec='red')
      plt.title('Número de casos do COVID-19\nLog dos Valores', fontsize=21)
    else:
      gdf.plot(ax=ax, color='darkred', markersize=gdf['Confirmed'], alpha=0.6, ec='red')
      plt.title('Número de casos do COVID-19\nValores Absolutos', fontsize=21)
      
    date = csv[:-4].split('-')
    plt.xlabel(date[1]+'/'+date[0]+'/'+date[2], fontsize=21)
    plt.xticks([],[])
    plt.yticks([],[])

    plt.text(-193,0,'@canalpeixebabel', fontsize=12, va='center', color='darkblue', rotation=90)
    plt.savefig(tipo+'/{:02d}.png'.format(k), dpi=150, format='png')

In [ ]:
for tipo in ['abs', 'log']:
  plt.figure(figsize=(20, 10))
  img = io.imread('{:s}/{:02d}.png'.format(tipo, len(os.listdir(root_path))))
  plt.imshow(img)
  plt.axis('off')
  plt.show()


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Capa Log
ax = world.plot(figsize=(15, 8), alpha=0.5)
plt.text(0, 0, 'Número de casos do COVID-19\nLog dos Valores', linespacing=2 ,ha='center', 
        bbox=dict(boxstyle="round",
                  ec=(1., 0.5, 0.5),
                  facecolor=(0.7, 0.1, 0.1, 0.8),
                  ),color='white', fontsize=30)
plt.text(-193,0,'@canalpeixebabel', fontsize=12, va='center', color='darkblue', rotation=90)
plt.savefig('logtitle.png', dpi=100,format='png')
plt.close('all')

# Capa Abs
ax = world.plot(figsize=(15, 8), alpha=0.5)
plt.text(0, 0, 'Número de casos do COVID-19\nValores Absolutos', linespacing=2 ,ha='center', 
        bbox=dict(boxstyle="round",
                  ec=(1., 0.5, 0.5),
                  facecolor=(0.7, 0.1, 0.1, 0.8),
                  ),color='white', fontsize=30)
plt.text(-193,0,'@canalpeixebabel', fontsize=12, va='center', color='darkblue', rotation=90)
plt.savefig('abstitle.png', dpi=100,format='png')
plt.close('all')

In [ ]:
os.environ['num_csv'] = str(len(os.listdir(root_path))-2)
!cp abs/$num_csv.png abs/`expr $num_csv + 1`.png
!cp abs/$num_csv.png abs/`expr $num_csv + 2`.png
!cp abs/$num_csv.png abs/`expr $num_csv + 3`.png

!cp log/$num_csv.png log/`expr $num_csv + 1`.png 
!cp log/$num_csv.png log/`expr $num_csv + 2`.png
!cp log/$num_csv.png log/`expr $num_csv + 3`.png

In [ ]:
import imageio
images = []
filenames = ['abstitle.png', 'abstitle.png', 'abstitle.png', 'abstitle.png', 'abstitle.png', 'abstitle.png', 'abstitle.png', 'abstitle.png']
filenames.extend(['abs/'+ f for f in sorted(os.listdir('abs/'))] )
filenames.extend( ['logtitle.png', 'logtitle.png', 'logtitle.png', 'logtitle.png', 'logtitle.png', 'logtitle.png', 'logtitle.png'] )
filenames.extend(['log/'+ f for f in sorted(os.listdir('log/'))] )
print(filenames)
for filename in filenames:
    if '.png' in filename:
      images.append(imageio.imread(filename))
imageio.mimsave('data-covid19.gif', images, duration=1/3)
print("data-covid19.gif salvo nos seus arquivos")